In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import string
from datetime import datetime as dt, timedelta
import matplotlib.pyplot as plt

fake = Faker() # Initialize Faker generator
Faker.seed(123)
np.random.seed(123)
random.seed(123)

#### sku_df 

In [3]:
import pandas as pd
import random

# Names generated by ChatGPT
action_figures_names = ["Neon Nova Knight", "Plasma Pulse Protector", "Galactic Gorilla Warrior", "Mystic Mind Master", "Stealth Strike Samurai",
                        "Bio Hazard Battler", "Astro Assassin Android", "Thunderstorm Titan", "Shadow Strike Sniper", "Arctic Avalanche Adventurer",
                        "Xeno Xplorer", "Chrono Crusader", "Ultimate Urban Warrior", "Savage Space Scavenger", "Infinity Ion Invader"]

plush_names = ["Cosmic Cuddly Comet", "Whisker Wizard Kitty", "Ninja Narwhal Nuzzler", "Galactic Gator Snuggler", "Pirate Parrot Pal",
               "Mystic Mermaid Hugger", "Adventurous Arctic Fox", "Space Explorer Sloth", "Enchanted Unicorn Snuggler", "Dino Discovery Dangler",
               "Robo Raccoon Cuddler", "Magical Monster Munchkin", "Superhero Squirrel Snuggler", "Cute Critter Collector", "Rainbow Robot Rabbit"]

puzzles_names = ["Galactic Gridlock", "Mystic Maze Master", "Enigma Expedition", "Space Saga Sudoku", "Pirate Puzzle Quest",
                 "Ninja Number Challenge", "Ancient Artifact Adventure", "Dino Dexterity Delight", "Robo-Logic Riddles", "Fantasy Fortress Fit",
                 "Monster Mystery Match", "Alien Abduction Assembly", "Superhero Sudoku Showdown", "Magical Mystery Map", "Cosmic Cube Conundrum"]

def generate_toy_products(num_products=15, perc_action_figures=0.55, perc_plush=0.30, perc_puzzles=0.15):
    
    # Calculate number of products for each category based on percentages
    num_action_figures = int(num_products * perc_action_figures)
    num_plush = int(num_products * perc_plush)
    num_puzzles = num_products - num_action_figures - num_plush

    # Ensure there are enough unique names
    if num_action_figures > len(action_figures_names) or num_plush > len(plush_names) or num_puzzles > len(puzzles_names):
        raise ValueError("Not enough unique names to satisfy the requested number of products.")

    # Generate lists for each category with unique names
    action_figures = [{'sku': f"A{100 + i}", # SKU start with A for Action Figures then increment
                       'sku_name': name, 
                       'product_category': "Action Figures"} 
                      for i, name in enumerate(random.sample(action_figures_names, num_action_figures))]
    
    plush = [{'sku': f"B{100 + i}", # SKU start with B for Plush toys then increment
              'sku_name': name, 
              'product_category': "Plush"} 
             for i, name in enumerate(random.sample(plush_names, num_plush))]
    
    puzzles = [{'sku': f"C{100 + i}", # SKU start with C for Puzzles then increment
                'sku_name': name, 
                'product_category': "Puzzles"} 
               for i, name in enumerate(random.sample(puzzles_names, num_puzzles))]

    # Concatenate lists into a single dataframe
    df = pd.DataFrame(action_figures + plush + puzzles)

    return df

# Generate toy products dataframe with num_products=20, perc_action_figures=0.6, perc_plush=0.3, perc_puzzles=0.1
sku_df = generate_toy_products(num_products = 20)
sku_df


,sku,sku_name,product_category
0,A100,Neon Nova Knight,Action Figures
1,A101,Stealth Strike Samurai,Action Figures
2,A102,Plasma Pulse Protector,Action Figures
3,A103,Astro Assassin Android,Action Figures
4,A104,Savage Space Scavenger,Action Figures
5,A105,Ultimate Urban Warrior,Action Figures
6,A106,Infinity Ion Invader,Action Figures
7,A107,Chrono Crusader,Action Figures
8,A108,Xeno Xplorer,Action Figures
9,A109,Thunderstorm Titan,Action Figures


#### bom_df

In [5]:
def generate_bom(skus_df):
    material_costs = {"plastic": 1.17, # Cost of material per 1 quantity
                      "wood": 0.5,      
                      "fabric": 0.79}
    
    part_counts = {'Action Figures': (5, 12),  # Number of parts based on product category
                   'Plush': (3, 8),
                   'Puzzles': (10, 30)}

    material_weights = {'Action Figures': [0.70, 0.15, 0.15], # Material type weights per product category [plastic, wood, fabric]
                        'Plush': [0.1, 0.1, 0.8],
                        'Puzzles': [0.1, 0.8, 0.1]}

    # Initialize lists for dataframe columns
    skus = []
    part_numbers = []
    quantities = []
    materials = []
    costs = []

    for idx, row in sku_df.iterrows(): # For each sku in sku_df
        num_parts = random.randint(*part_counts[row['product_category']]) # Select random number of parts with ranges based on product category

        for part in range(num_parts): # For each part in sku, generate each attribute
            skus.append(row['sku'])
            part_numbers.append(''.join(random.choices(string.ascii_uppercase + string.digits, k=3)))  # 3 Random digits and letters
            quantities.append(random.randint(1, 5))  # Random quantity between 1 and 5
            material = random.choices(["plastic", "wood", "fabric"], weights = material_weights[row['product_category']], k=1)[0] # Random material type based on product category weights
            materials.append(material)
            costs.append(material_costs[material])  # Assign cost based on material

    # Create dataframe using the lists
    bom_df = pd.DataFrame({
        'sku': skus,
        'part_number': part_numbers,
        'quantity': quantities,
        'material': materials,
        'cost': costs
    }).assign(cost_ext=lambda df: df['quantity'] * df['cost'])  # Calculate total cost, quantity * cost per part

    return bom_df

sku_df = generate_toy_products(num_products=25)  # Generate toy products dataframe, sku_df
bom_df = generate_bom(sku_df) # Generate bill of materials dataframe, bom_df based on sku_df
bom_df

,sku,part_number,quantity,material,cost,cost_ext
0,A100,M2H,5,plastic,1.17,5.85
1,A100,U8A,5,plastic,1.17,5.85
2,A100,T2V,4,plastic,1.17,4.68
3,A100,PS4,2,wood,0.50,1.00
4,A100,CRJ,3,plastic,1.17,3.51
...,...,...,...,...,...,...
227,C104,OL1,5,wood,0.50,2.50
228,C104,K5V,1,wood,0.50,0.50
229,C104,DYE,4,wood,0.50,2.00
230,C104,YQ8,3,wood,0.50,1.50


#### store_df

In [7]:
# List of randomly generated toy store names from ChatGPT
store_names = ["Playful Pals Toy Emporium", "Whimsy Wonderland Toys", "Marvelous Miniatures", "ToyBox Treasures", "Giggle Gadget Toys", "Dreamy Dollhouse Delights", 
               "Twinkle Town Toys", "Magic Moments Toy Shop", "Starlight Stuffed Animals", "Playland Paradise", "Toy Treasure Trove", "Giggles & Games", 
               "WonderToys", "The Fun Emporium", "Kiddie Kingdom", "Toybox Universe", "Playtime Junction", "Happy Hands Toys", "Magic Toy Chest", "Little Explorers", 
               "Jolly Joy Toys", "Playful Planet", "The Toy Loft", "Toy Haven", "Whimsical Wonders", "Playdate Palace", "Tiny Town Toys", "Toy Safari", ]

# US regions dictionary
regions = {
    'Northeast': ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'New Jersey', 'Pennsylvania'],
    'Midwest': ['Ohio', 'Michigan', 'Indiana', 'Wisconsin', 'Illinois', ],
                #'Minnesota', 'Iowa', 'Missouri', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas'],
    'South': ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', ],
              #'Kentucky', 'Tennessee',  'Mississippi', 'Alabama', ],
              #'Oklahoma', 'Texas', 'Arkansas', 'Louisiana'],
    #'West': ['Montana', 'Idaho', 'Wyoming', 'Colorado', 'New Mexico', 'Arizona', 'Utah', 'Nevada', 'California', 'Oregon', 'Washington']
}

# Generate a random number of salespeople names (between 2 and 4)
def generate_salespeople_names():
    num_salespeople = np.random.randint(2, 5)
    salespeople_names = [fake.name() for _ in range(num_salespeople)]
    return salespeople_names

# Generate region per state
def get_region(state):
    for region, states in regions.items():
        if state in states:
            return region
    return 'Unknown'

# Store salesperson names per region in a dictionary
salespeople_by_region = {region: generate_salespeople_names() for region in regions}

def generate_store_data(num_stores):
    states = random.sample(sum(regions.values(), []), len(sum(regions.values(), [])))
    states_list = states * (num_stores // len(states) + 1) # create a list of all states and assign store a state to ensure that each state is represented
    store_data = (
        pd.DataFrame({
            'store_name': [random.choice(store_names) for _ in range(num_stores)]
        })
        .assign(
            store_id = lambda df: [''.join(random.choices(string.ascii_uppercase + string.digits, k=6)) for _ in range(num_stores)], # Random assortment of 6 numbers and letters
            store_state = lambda df: [states_list[i] for i in range(num_stores)], # Iterate through state to ensure each state is represented
            store_region = lambda df: df['store_state'].map(get_region), # Get region by state
            sales_person = lambda df: df['store_region'].map(lambda region: random.choice(salespeople_by_region[region])) # Random person from salesperson-region dictionary
        )
    )
    return store_data

store_df = generate_store_data(75)
store_df


,store_name,store_id,store_state,store_region,sales_person
0,Twinkle Town Toys,74A5I8,Michigan,Midwest,Jerome Whitehead
1,Toy Safari,Y81SMY,Rhode Island,Northeast,Evelyn Christian
2,Playful Planet,5D49VH,Florida,South,Ryan Mack
3,Dreamy Dollhouse Delights,WVMSAG,Illinois,Midwest,Jerome Whitehead
4,Playtime Junction,HKGA4L,Ohio,Midwest,Kyle Jones
...,...,...,...,...,...
70,Dreamy Dollhouse Delights,5Y2LNA,Rhode Island,Northeast,Evelyn Christian
71,Playdate Palace,NQT4BZ,Florida,South,Thomas Berry
72,Happy Hands Toys,98PRXW,Illinois,Midwest,Kyle Jones
73,Toy Safari,88P1VX,Ohio,Midwest,Aaron Graham


#### price_df

In [9]:
def store_and_sku_df():
    return pd.merge(store_df[['store_id']], sku_df[['sku']], how = 'cross')

def cost_per_sku_df():
    return \
    (bom_df.groupby('sku')
     .agg(cost = ('cost_ext', 'sum'))
     .reset_index()
    )
     
def generate_price_df():
    return \
    (pd.merge(
        store_and_sku_df(),
        cost_per_sku_df())
     .assign(
         multiplier = lambda df: df.apply(lambda _: round(random.uniform(0.95, 1.52), 2), axis = 1),
         price = lambda df: round(df['cost'] * df['multiplier'], 2)
         )
    )

price_df = generate_price_df()
price_df

,store_id,sku,cost,multiplier,price
0,74A5I8,A100,30.41,1.23,37.40
1,74A5I8,A101,23.52,1.01,23.76
2,74A5I8,A102,24.23,1.12,27.14
3,74A5I8,A103,41.80,1.47,61.45
4,74A5I8,A104,22.16,1.28,28.36
...,...,...,...,...,...
1870,Z3R82N,C100,28.19,1.01,28.47
1871,Z3R82N,C101,45.63,1.43,65.25
1872,Z3R82N,C102,24.96,1.38,34.44
1873,Z3R82N,C103,31.68,1.47,46.57


#### sales_df

In [11]:
def generate_orders(num_orders):
    orders = []

    start_date = dt(2023, 1, 1)
    end_date = dt(2023, 12, 31)
    delta = end_date - start_date
    
    for i in range(num_orders):
        order_number = 10000 + i + 1
        order_date = start_date + timedelta(days = np.random.randint(delta.days + 1))
        num_skus = np.random.randint(1, 10)  # Random number of SKUs per order (1 to 3)
        store_id = store_df['store_id'].sample().iloc[0]
        
        skus_in_order = set()  # To track unique SKUs in the current order
        
        for j in range(num_skus):
            available_skus = list(set(sku_df['sku']) - skus_in_order)
            
            if not available_skus:
                break
            
            sku = np.random.choice(available_skus)  # Randomly select from available SKUs
            skus_in_order.add(sku)  # Add SKU to set of SKUs in current order
            
            quantity = np.random.randint(5, 35)  # Random quantity per sku between 5 and 9
            # price = price_df[(price_df['store_id'] == store_id) & (price_df['sku'] == sku)]['price'].iloc[0]
            # price_ext = price * quantity
            
            orders.append({
                'order_id': order_number,
                'order_date': order_date,
                'sku': sku,
                'quantity': quantity,
                'store_id': store_id,
                # 'price_ea': price,
                # 'price_ext': price_ext
            })
    sales_df = (pd.DataFrame(orders)
                .merge(price_df[['store_id', 'sku','price']], on=['store_id', 'sku'], how='left')
               )

    
    return sales_df

In [12]:
sales_23_df = generate_orders(40_000)

In [13]:
sales_24_df = \
(sales_23_df
 .query("order_date.dt.month <= 6")
 .assign(
     order_date = lambda df: df.order_date + pd.DateOffset(years=1),
     quantity = lambda df: round(df['quantity'] * np.random.uniform(0.93, 1.10, size=len(df)), 0).astype('int64'),
     price = lambda df: round(df['price'] * np.random.uniform(0.98, 1.05, size=len(df)), 0).astype('int64'),
 )
)
sales_24_df
 
sales_df = \
(pd.concat([sales_23_df, sales_24_df], ignore_index = True)
 .assign(price_ext = lambda df: df.price * df.quantity)
)
sales_df

,order_id,order_date,sku,quantity,store_id,price,price_ext
0,10001,2023-11-19,A111,14,W4R6TK,18.84,263.76
1,10001,2023-11-19,A100,8,W4R6TK,41.97,335.76
2,10001,2023-11-19,A105,8,W4R6TK,16.02,128.16
3,10002,2023-09-01,C102,28,5D49VH,35.69,999.32
4,10002,2023-09-01,A107,6,5D49VH,17.73,106.38
...,...,...,...,...,...,...,...
298846,49999,2024-02-28,A103,7,12QLCH,44.00,308.00
298847,50000,2024-06-12,C100,19,N7W6LE,42.00,798.00
298848,50000,2024-06-12,B106,30,N7W6LE,13.00,390.00
298849,50000,2024-06-12,A103,11,N7W6LE,52.00,572.00


In [14]:
(sales_df
 .groupby(pd.Grouper(key = 'order_date', freq = 'MS'))
 .agg(sales = ('price_ext', 'sum'))
 .assign(
     year = lambda df: df.index.year,
     month = lambda df: df.index.month
 )
 .reset_index(drop = True)
 .groupby(['year', 'month'])
 .agg(sales = ('sales', 'sum'))
 .unstack(level = 'year')
 .query("month <= 6")
 .sum()
)

       year
sales  2023    53688226.09
       2024    55332329.00
dtype: float64

#### target_df

In [16]:
target_df_over = \
(sales_23_df
 .assign(month = lambda df: df.order_date.dt.month)
 .query("month in [1, 2, 4]")
 .drop(columns = ("month"))
 .groupby(['store_id', 'sku', pd.Grouper(key = 'order_date', freq = 'MS')])
 .agg(
     quantity = ('quantity', 'sum'),
     sales = ('price', 'sum')
 )
 .reset_index()
 .merge(price_df[['store_id', 'sku','price']], on=['store_id', 'sku'], how='left')
 .assign(
     order_date = lambda df: df.order_date + pd.DateOffset(years = 1),
     target_quantity = lambda df: round(df['quantity'] * np.random.uniform(0.95, 1.15, size = len(df)), 0),
     target_price = lambda df: round(df['price'] * np.random.uniform(0.98, 1.10, size = len(df)), 2),
     target_sales = lambda df: round(df['target_quantity'] * df['target_price'], 2)
 )
)
target_df_under = \
(sales_23_df
 .assign(month = lambda df: df.order_date.dt.month)
 .query("month in [3, 5, 6]")
 .drop(columns = ("month"))
 .groupby(['store_id', 'sku', pd.Grouper(key = 'order_date', freq = 'MS')])
 .agg(
     quantity = ('quantity', 'sum'),
     sales = ('price', 'sum')
 )
 .reset_index()
 .merge(price_df[['store_id', 'sku','price']], on=['store_id', 'sku'], how='left')
 .assign(
     order_date = lambda df: df.order_date + pd.DateOffset(years = 1),
     target_quantity = lambda df: round(df['quantity'] * np.random.uniform(0.92, 1.06, size = len(df)), 0),
     target_price = lambda df: round(df['price'] * np.random.uniform(0.99, 1.02, size = len(df)), 2),
     target_sales = lambda df: round(df['target_quantity'] * df['target_price'], 2)
 )
)
target_df = pd.concat([target_df_over, target_df_under], ignore_index = True)
target_df

,store_id,sku,order_date,quantity,sales,price,target_quantity,target_price,target_sales
0,0MMWJC,A100,2024-01-01,122,405.09,45.01,130.0,48.91,6358.30
1,0MMWJC,A100,2024-02-01,257,630.14,45.01,281.0,45.97,12917.57
2,0MMWJC,A100,2024-04-01,137,360.08,45.01,140.0,49.06,6868.40
3,0MMWJC,A101,2024-01-01,150,302.67,33.63,158.0,34.35,5427.30
4,0MMWJC,A101,2024-02-01,120,201.78,33.63,121.0,36.37,4400.77
...,...,...,...,...,...,...,...,...,...
11242,Z3R82N,C103,2024-05-01,196,419.13,46.57,206.0,47.19,9721.14
11243,Z3R82N,C103,2024-06-01,103,279.42,46.57,100.0,47.14,4714.00
11244,Z3R82N,C104,2024-03-01,150,456.12,50.68,156.0,51.14,7977.84
11245,Z3R82N,C104,2024-05-01,135,354.76,50.68,133.0,50.76,6751.08


In [17]:
# target_df = \
# (sales_df
#  .query("order_date.dt.year == 2024")
#  .groupby(['store_id', 'sku', pd.Grouper(key = 'order_date', freq = 'MS')])
#  .agg(
#      quantity = ('quantity', 'sum'),
#      sales = ('price_ext', 'sum')
#  )
#  .reset_index()
#  .assign(
#      order_date = lambda df: df.order_date + pd.DateOffset(years = 1),
#      multiplier = lambda df: np.random.uniform(0.80, 1.20, size = len(df)),
#      target_quantity = lambda df: round(df['quantity'] * df['multiplier'], 0).astype('int64'),
#      target_sales = lambda df: round(df['sales'] * df['multiplier'], 2)
#  )
# )
# target_df

#### financials_df

In [19]:
def group_actual():
    return \
    (sales_df
     .query("order_date.dt.year == 2024")
     .groupby([pd.Grouper(key = 'order_date', freq = 'MS'), 'store_id', 'sku'])
     .agg(
         actual_quantity = ('quantity', 'sum'),
         actual_sales = ('price_ext', 'sum')
     )
     .reset_index()
    )

def group_target():
    return \
    (target_df 
     .groupby([pd.Grouper(key = 'order_date', freq = 'MS'), 'store_id', 'sku'])
     .agg(
         target_quantity = ('target_quantity', 'sum'),
         target_sales = ('target_sales', 'sum')
     )
     .reset_index()
    )

def group_py():
    return \
    (sales_df
     .query("order_date.dt.year == 2023")
     .assign(order_date = lambda df: df['order_date'] + pd.DateOffset(years=1))
     .groupby([pd.Grouper(key = 'order_date', freq = 'MS'), 'store_id', 'sku'])
     .agg(
         py_quantity=('quantity', 'sum'),
         py_sales=('price_ext', 'sum'))    
     .reset_index()   
    )

def sku_cost():
    return \
    (bom_df
     .groupby('sku')
     .agg(total_cost = ('cost_ext', 'sum'))
     .reset_index()
    )

def generate_financials_df():
    return \
    (pd.merge(group_actual(), group_target(), how = 'outer')
     .merge(group_py(), how = 'outer')
     .merge(sku_cost(), how = 'outer')
     .assign(
         actual_cost = lambda df: df.actual_quantity * df.total_cost,
         py_cost = lambda df: df.py_quantity * df.total_cost,
         target_cost = lambda df: df.target_quantity * df.total_cost,
         actual_margin = lambda df: df.actual_sales - df.actual_cost,
         py_margin = lambda df: df.py_sales - df.py_cost,
         target_margin = lambda df: df.target_sales - df.target_cost
     )
    )
    
financials_df = generate_financials_df()
financials_df

,order_date,store_id,sku,actual_quantity,actual_sales,target_quantity,target_sales,py_quantity,py_sales,total_cost,actual_cost,py_cost,target_cost,actual_margin,py_margin,target_margin
0,2024-01-01,0MMWJC,A100,129.0,5898.0,130.0,6358.30,122,5491.22,30.41,3922.89,3710.02,3953.30,1975.11,1781.20,2405.00
1,2024-01-01,12QLCH,A100,198.0,5823.0,199.0,6178.95,193,5575.77,30.41,6021.18,5869.13,6051.59,-198.18,-293.36,127.36
2,2024-01-01,130QJN,A100,106.0,3951.0,104.0,4019.60,101,3655.19,30.41,3223.46,3071.41,3162.64,727.54,583.78,856.96
3,2024-01-01,1RCWDN,A100,154.0,5873.0,170.0,6922.40,154,5713.40,30.41,4683.14,4683.14,5169.70,1189.86,1030.26,1752.70
4,2024-01-01,27QOA7,A100,307.0,11092.0,292.0,11230.32,298,10602.84,30.41,9335.87,9062.18,8879.72,1756.13,1540.66,2350.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22492,2024-12-01,XSKTYJ,C104,NaN,NaN,NaN,NaN,165,6022.50,35.44,NaN,5847.60,NaN,NaN,174.90,NaN
22493,2024-12-01,Y81SMY,C104,NaN,NaN,NaN,NaN,183,8430.81,35.44,NaN,6485.52,NaN,NaN,1945.29,NaN
22494,2024-12-01,YAK8XB,C104,NaN,NaN,NaN,NaN,188,6596.92,35.44,NaN,6662.72,NaN,NaN,-65.80,NaN
22495,2024-12-01,Z37T6G,C104,NaN,NaN,NaN,NaN,149,6706.49,35.44,NaN,5280.56,NaN,NaN,1425.93,NaN


#### Save

In [21]:
sku_df.to_csv(r'C:\Users\cneva\Projects\Toy Sales\Datasets\sku.csv', index = False)
bom_df.to_csv(r'C:\Users\cneva\Projects\Toy Sales\Datasets\bom.csv', index = False)
store_df.to_csv(r'C:\Users\cneva\Projects\Toy Sales\Datasets\store.csv', index = False)
price_df.to_csv(r'C:\Users\cneva\Projects\Toy Sales\Datasets\price.csv', index = False)
sales_df.to_csv(r'C:\Users\cneva\Projects\Toy Sales\Datasets\sales.csv', index = False)
target_df.to_csv(r'C:\Users\cneva\Projects\Toy Sales\Datasets\target.csv', index = False)
financials_df.to_csv(r'C:\Users\cneva\Projects\Toy Sales\Datasets\financials.csv', index = False)